In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pickle
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input,VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'distutils'

In [ ]:
WORKING_DIR = '/content/drive/MyDrive/COLAB/working'

In [ ]:
# Load features from pickle
with open(os.path.join(WORKING_DIR, 'features.pkl'), 'rb') as f:
    features = pickle.load(f)



In [ ]:
with open(os.path.join(WORKING_DIR, 'tokenizer.pkl'), 'rb') as t:
    tokenizer = pickle.load(t)

In [ ]:
# Extract Image Features
# Load VGG19 Model
model = VGG19(weights=os.path.join(WORKING_DIR, 'vgg19_weights_tf_dim_ordering_tf_kernels.h5'))

# Restructure model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
# Path to the model file in Google Drive
model_path = os.path.join(WORKING_DIR, 'best_model.h5')

# Load the model
loaded_model = load_model(model_path)

In [ ]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


In [ ]:
# Preprocess the new image
# Assume 'new_image' is the dynamic image
# Preprocess the image
'''new_image = load_img(new_image_path, target_size=(224, 224))
new_image = img_to_array(new_image)
new_image = new_image.reshape((1, new_image.shape[0], new_image.shape[1], new_image.shape[2]))
new_image = preprocess_input(new_image)

# Extract features using the pre-trained model
new_feature = model.predict(new_image, verbose=0)'''

# Define a function to generate captions for dynamic images
def generate_caption(model, tokenizer, max_length, image_feature):
    in_text = 'startseq'
    for _ in range(max_length):
        # Encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        # Predict next word
        yhat = model.predict([image_feature, sequence], verbose=0)
        # Convert probability to integer
        yhat = np.argmax(yhat)
        # Map integer to word
        word = word_for_id(yhat, tokenizer)
        # Stop if we cannot map the word
        if word is None:
            break
        # Append as input for generating the next word
        in_text += ' ' + word
        # Stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text



In [ ]:
def image_feature(new_image_path,model):
  new_image = load_img(new_image_path, target_size=(224, 224))
  new_image = img_to_array(new_image)
  new_image = new_image.reshape((1, new_image.shape[0], new_image.shape[1], new_image.shape[2]))
  new_image = preprocess_input(new_image)

  # Extract features using the pre-trained model
  new_feature = model.predict(new_image, verbose=0)
  return new_feature

In [ ]:
image_path = os.path.join(WORKING_DIR, 'image2.jpg')
new_feature = image_feature(image_path, model)

In [ ]:
#Generate caption for the new image
caption = generate_caption(loaded_model, tokenizer, 35, new_feature)
print(caption)


startseq person skiing down snowy mountain endseq
